In [ ]:
# %pip install gensim --user

In [2]:
import gensim

In [3]:
import sys, os
# sys.path.insert(0, '/datasets/home/51/451/yuz530/.local/lib/python3.7/site-packages/')

In [4]:
!ls .

ABA.cor  data  jeff_word2vec.ipynb  test.cor


In [5]:
import numpy as np
from scipy import sparse

In [6]:
A = sparse.load_npz('./data/A_200.npz')
B = sparse.load_npz('./data/B_200.npz')
P = sparse.load_npz('./data/P.npz')
A_csr = A
A_csc = A.tocsc(copy=True)  # memory is cheap ;D

In [7]:
# ABPBA
def ABPBA():
    while True:
        app_i = np.random.choice(np.arange(A.shape[0]))
        api_i = np.random.choice(np.nonzero(A_csr[app_i])[1])
        api_bi = np.random.choice(np.nonzero(B[:, api_i])[0])
        api_p = np.random.choice(np.nonzero(P[:, api_bi])[0])
        api_bj = np.random.choice(np.nonzero(B[:, api_p])[0])
        app_j = np.random.choice(np.nonzero(A_csc[:, api_bj])[0])

        yield f'app_{app_i} api_{api_i} api_{api_bi} api_{api_p} api_{api_bj} app_{app_j}'
        
# ABA
def ABA():
    while True:
        app_i = np.random.choice(np.arange(A.shape[0]))
        api_i = np.random.choice(np.nonzero(A_csr[app_i])[1])
        api_bi = np.random.choice(np.nonzero(B[:, api_i])[0])
        app_j = np.random.choice(np.nonzero(A_csc[:, api_bi])[0])

        yield f'app_{app_i} api_{api_i} api_{api_bi} app_{app_j}'
        
# it = iter_neighbors()

In [8]:
app_i = np.random.choice(np.arange(A.shape[0]))
print(app_i)
api_i = np.random.choice(np.nonzero(A_csr[app_i])[1])
print(api_i)
api_bi = np.random.choice(np.nonzero(B[:, api_i])[0])
print(api_bi)
if(api_bi > A.shape[1]):
    api_bi = A.shape[1]-1
app_j = np.random.choice(np.nonzero(A_csc[:, api_bi])[0])
app_j

82
13185
698097


73

In [9]:
(B[:, api_i])

<916573x1 sparse matrix of type '<class 'numpy.int64'>'
	with 3553 stored elements in Compressed Sparse Row format>

In [10]:
# %lprun -f iter_neighbors a = [next(it) for i in range(10000)] 

In [11]:
#note ValueError: 'a' cannot be empty unless no samples are taken B col all 0 why?

for _ in range(10):
    print(next(ABA()) + '\n')

app_167 api_155032 api_854512 app_128

app_101 api_826302 api_9064 app_79

app_90 api_297228 api_297007 app_32

app_130 api_859387 api_859402 app_130

app_13 api_225915 api_34280 app_56

app_188 api_850373 api_504 app_11

app_60 api_539450 api_9550 app_80

app_68 api_499715 api_13712 app_158



ValueError: 'a' cannot be empty unless no samples are taken

In [73]:
%%time
f = open('ABA.cor', 'w')
for _ in range(10000):
    try:
        f.write(next(ABA()) + '\n')
    except:
        pass
f.close()

CPU times: user 8min 2s, sys: 955 ms, total: 8min 3s
Wall time: 8min 3s


In [74]:
# f = open('test.cor', 'w')
# for _ in range(10000):
#     f.write(next(it) + '\n')
# f.close()

In [75]:
!head ABA.cor

app_159 api_108546 api_110272 app_32
app_51 api_48767 api_253470 app_17
app_80 api_400475 api_413947 app_54
app_8 api_167195 api_167194 app_8
app_179 api_839047 api_3357 app_124
app_143 api_826298 api_826297 app_173
app_60 api_539320 api_539157 app_60
app_8 api_174864 api_174861 app_8
app_91 api_7137 api_879458 app_142
app_101 api_826290 api_892119 app_164


In [76]:
%%time
from gensim import utils
import gensim.models

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = 'ABA.cor'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
#             yield utils.simple_preprocess(line)
            yield line.strip().split(' ')

CPU times: user 150 µs, sys: 0 ns, total: 150 µs
Wall time: 162 µs


In [77]:
sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences, min_count=1, size=200)

In [78]:
for i, word in enumerate(model.wv.vocab):
    if i == 10:
        break
    print(word)

app_159
api_108546
api_110272
app_32
app_51
api_48767
api_253470
app_17
app_80
api_400475


In [86]:
model.wv.similarity("app_177", "app_32")

0.92201227

In [85]:
print(model.wv.most_similar(positive=['app_177'], topn=5))

[('app_142', 0.9344480037689209), ('app_2', 0.9337429404258728), ('app_46', 0.9320647120475769), ('app_65', 0.9311726689338684), ('app_50', 0.9309657216072083)]


In [ ]:
%pip install Cython

In [ ]:
from gensim.models.word2vec import FAST_VERSION

In [ ]:
FAST_VERSION